*Verified by Leila Belabbassi (to work with Pangeo), July 12, 2018*

In [2]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd
import re
import requests
import pickle as pk
import os
import gc
#!pip install netcdf4 == 1.5.0

### Start a Dask Cluster

In [3]:
# Sensor E: url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T104012-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# Sensor B url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181020T213838-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample/catalog.html'
# Sensor C url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T041943-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html'
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191015T040958136Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample/catalog.html'

In [4]:
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i + '#fillmismatch') for i in x]
#datasets

In [5]:
from dask.distributed import Client

client = Client("tcp://10.0.128.163:43917")
client

OSError: Timed out trying to connect to 'tcp://10.0.128.163:43917' after 10 s: Timed out trying to connect to 'tcp://10.0.128.163:43917' after 10 s: in <distributed.comm.tcp.TCPConnector object at 0x7fe2a7eaa908>: ConnectionRefusedError: [Errno 111] Connection refused

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/client.py", line 1284, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/client.py", line 1013, in _reconnect
    await self._close()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/client.py", line 1284, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures

## origional loop

In [ ]:
num = 0
for i in datasets:
    ds = xr.open_dataset(i)
    ds = ds.swap_dims({'obs': 'time'})

    pressure_min = pd.DataFrame()
    pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
    del pressure_min.index.name

    pressure_min = pressure_min.dropna()

    out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled' + '.pd'
    num = num +1

    with open(out, 'wb') as fh:
        pk.dump(pressure_min,fh)

    gc.collect()

# pd routine

In [ ]:
%%time
for i in datasets:
    ds = xr.open_dataset(i)
    ds = ds.swap_dims({'obs': 'time'})

    pressure_min = pd.DataFrame()
    pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
    del pressure_min.index.name

    pressure_min = pressure_min.dropna()

    out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled' + '.nc'
    ds = xr.Dataset.from_dataframe(pressure_min)
    ds.to_netcdf(out)

# Test Loop PD

In [ ]:
%%time
i=datasets[1]
ds = xr.open_dataset(i)
ds = ds.swap_dims({'obs': 'time'})

pressure_min = pd.DataFrame()
pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
del pressure_min.index.name

pressure_min = pressure_min.dropna()
out = 'minute_mean_data/test.nc'

ds = xr.Dataset.from_dataframe(pressure_min)
ds.to_netcdf(out)

## xr routine 

In [1]:
from dask.distributed import Client

client = Client("tcp://10.0.128.163:39775")
client

Client Scheduler: tcp://10.0.128.163:39775 Dashboard: /user/daxsoule/proxy/8787/status,Cluster Workers: 20 Cores: 20 Memory: 25.00 GB


In [ ]:
%%time
for i in datasets:
    ds = xr.open_mfdataset([i], parallel =True, combine='by_coords')
    ds = ds.swap_dims({'obs': 'time'})
    ds= ds['bottom_pressure'].to_dataset()
    ds=ds.resample(time='T').mean()
    out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled_test' + '.nc'
    ds.to_netcdf(out)

In [ ]:
ds = xr.open_mfdataset([datasets[0]], parallel =True)

In [ ]:
ds['bottom_pressure'].to_netcdf?

In [ ]:
ds['bottom_pressure'].to_dataset()

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.128.163:39659")
client

In [ ]:
%%time
i= datasets[1]
ds = xr.open_mfdataset([i], parallel=True, drop_variables= ['deployment', 'id', 'bottom_pressure_qc_executed', 'date_time_string'
                                                  , 'driver_timestamp', 'ingestion_timestamp', 'internal_timestamp'
                                                  ,'port_timestamp', 'preferred_timestamp', 'press_trans_temp'
                                                  ,'provenance','sensor_id', 'time_sync_flag', 'bottom_pressure_qc_results']
                            )
ds=ds.swap_dims({'obs': 'time'}).resample(time='T').mean()
out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled' + '.nc'
ds.to_netcdf(out)

In [ ]:
import netcdf4

In [ ]:
#!conda list

In [ ]:
netcdf.__version__

In [ ]:
!rm minute_mean_data/*

In [ ]:
def non_time_coords(ds):
    return [v for v in ds.data_vars
            if 'time' not in ds[v].dims]

def drop_non_essential_vars_pop(ds):
    return ds.drop(non_time_coords(ds))

In [ ]:
ds = xr.open_mfdataset(datasets[0:1], parallel=True, data_vars({'time', 'bottom_pressure'}),combine='by_coords')
#ds = ds.swap_dims({'obs': 'time'}.resample(time='10T').mean()
ds

In [ ]:
ds = xr.open_mfdataset(datasets[0:1], parallel=True,combine='by_coords')
#ds = ds.swap_dims({'obs': 'time'}.resample(time='10T').mean()
ds

In [ ]:
ds = xr.open_dataset(datasets[1], drop_variables= ['deployment', 'id', 'bottom_pressure_qc_executed', 'date_time_string'
                                                  , 'driver_timestamp', 'ingestion_timestamp', 'internal_timestamp'
                                                  ,'port_timestamp', 'preferred_timestamp', 'press_trans_temp'
                                                  ,'provenance','sensor_id', 'time_sync_flag', 'bottom_pressure_qc_results'])
ds=ds.swap_dims({'obs': 'time'}).resample(time='T').mean()
ds.to_netcdf('minute_mean_data/saved_on_disk.nc')

In [ ]:
ds = xr.open_dataset(datasets[1],decode_times=False)
ds=ds.swap_dims({'obs': 'time'}).resample(time='T').mean()
ds=ds['bottom_pressure']
ds.to_netcdf('minute_mean_data/saved_on_disk.nc')

In [ ]:
xr.open_dataset?

In [ ]:
ds=xr.open_dataset(datasets[1])

In [ ]:
ds

In [ ]:
df = ds.to_dask_dataframe()

In [ ]:
df.head()